In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Data Preparation
train_df = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv')

# Split data into training, validation, and testing sets
train_data, test_data = train_test_split(train_df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Data preprocessing functions (e.g., resize, normalize)

img_height = 224
img_width = 224
batch_size = 32
epochs =5 
# Step 2: Model Building
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 3: Model Training
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train/',
    x_col='file_id',
    y_col='age',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train/',
    x_col='file_id',
    y_col='age',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train/',
    x_col='file_id',
    y_col='age',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False)

history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Step 4: Model Evaluation
loss = model.evaluate(test_generator)
print("Test Loss:", loss)

# Step 5: Deployment
# Save the model for future use
model.save('age_prediction_model.h5')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Found 13657 validated image filenames.
Found 3415 validated image filenames.
Found 4268 validated image filenames.
Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/427 ━━━━━━━━━━━━━━━━━━━━ 1:45:24 15s/step - loss: 1257.7145

I0000 00:00:1712609342.890915      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1712609342.908709      89 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


427/427 ━━━━━━━━━━━━━━━━━━━━ 0s 529ms/step - loss: 389.2455

W0000 00:00:1712609569.206529      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


427/427 ━━━━━━━━━━━━━━━━━━━━ 258s 571ms/step - loss: 389.1262 - val_loss: 272.7801
Epoch 2/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 187s 431ms/step - loss: 297.4511 - val_loss: 247.9888
Epoch 3/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 187s 431ms/step - loss: 285.9945 - val_loss: 254.6792
Epoch 4/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 186s 429ms/step - loss: 264.4639 - val_loss: 251.9907
Epoch 5/5
427/427 ━━━━━━━━━━━━━━━━━━━━ 187s 431ms/step - loss: 256.0478 - val_loss: 209.2642
134/134 ━━━━━━━━━━━━━━━━━━━━ 19s 145ms/step - loss: 202.5833
Test Loss: 202.43115234375


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define your CNN model
class AgePredictionModel(nn.Module):
    def __init__(self):
        super(AgePredictionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 28 * 28, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define a custom dataset class
class AgeDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        image = Image.open(self.root_dir + img_name)
        age = self.data.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, age

# Load data and split into train, val, and test sets
data = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv')
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and dataloaders
train_dataset = AgeDataset(csv_file='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv', root_dir='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train/', transform=transform)
val_dataset = AgeDataset(csv_file='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv', root_dir='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train/', transform=transform)
test_dataset = AgeDataset(csv_file='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv', root_dir='/kaggle/input/smai-24-age-prediction/content/faces_dataset/train/', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss function, and optimizer
model = AgePredictionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10
# Train the model
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # Validate the model
    model.eval()
    val_loss = 0.0
    val_mae = 0.0
    val_rmse = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            val_loss += loss.item() * inputs.size(0)
            val_mae += mean_absolute_error(labels.cpu().numpy(), outputs.cpu().numpy())
            val_rmse += mean_squared_error(labels.cpu().numpy(), outputs.cpu().numpy(), squared=False)
    val_loss /= len(val_loader.dataset)
    val_mae /= len(val_loader.dataset)
    val_rmse /= len(val_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val MAE: {val_mae:.4f}, Val RMSE: {val_rmse:.4f}')

# Test the model
model.eval()
test_loss = 0.0
test_mae = 0.0
test_rmse = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        test_loss += loss.item() * inputs.size(0)
        test_mae += mean_absolute_error(labels.cpu().numpy(), outputs.cpu().numpy())
        test_rmse += mean_squared_error(labels.cpu().numpy(), outputs.cpu().numpy(), squared=False)
test_loss /= len(test_loader.dataset)
test_mae /= len(test_loader.dataset)
test_rmse /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}, Test RMSE: {test_rmse:.4f}')

Epoch [1/10], Train Loss: 249.3024, Val Loss: 196.0022, Val MAE: 0.3191, Val RMSE: 0.3790
Epoch [2/10], Train Loss: 151.0959, Val Loss: 117.2479, Val MAE: 0.2460, Val RMSE: 0.3015
Epoch [3/10], Train Loss: 114.9866, Val Loss: 93.1071, Val MAE: 0.2307, Val RMSE: 0.2855
Epoch [4/10], Train Loss: 93.9735, Val Loss: 79.2654, Val MAE: 0.2122, Val RMSE: 0.2674
Epoch [5/10], Train Loss: 77.5517, Val Loss: 73.1018, Val MAE: 0.2024, Val RMSE: 0.2558
Epoch [6/10], Train Loss: 67.5468, Val Loss: 54.7141, Val MAE: 0.1713, Val RMSE: 0.2140
Epoch [7/10], Train Loss: 60.1001, Val Loss: 58.6312, Val MAE: 0.1857, Val RMSE: 0.2343
Epoch [8/10], Train Loss: 50.8804, Val Loss: 39.8895, Val MAE: 0.1494, Val RMSE: 0.1898
Epoch [9/10], Train Loss: 42.3905, Val Loss: 32.7313, Val MAE: 0.1359, Val RMSE: 0.1712
Epoch [10/10], Train Loss: 35.8799, Val Loss: 29.4340, Val MAE: 0.1303, Val RMSE: 0.1658
Test Loss: 29.4340, Test MAE: 0.1303, Test RMSE: 0.1658


In [6]:
torch.save(model.state_dict(), 'age_prediction_model_pytorch.pth')

In [7]:
torch.save(model.state_dict(), 'age_prediction_model_pytorch.pt')

In [14]:
import torch
from torchvision import transforms
from PIL import Image

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the image
image_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test/image_1002.jpg'  # Change this to the path of your sample image
image = Image.open(image_path)

# Preprocess the image
input_image = transform(image).unsqueeze(0)  # Add batch dimension

# Load the model
model = AgePredictionModel().to(device)
model.load_state_dict(torch.load('/kaggle/working/age_prediction_model_pytorch.pth'))  # Load the trained model

# Set model to evaluation mode
model.eval()

# Move input image to GPU if available
input_image = input_image.to(device)

# Get the prediction
with torch.no_grad():
    output = model(input_image)

predicted_age = output.item()
print("Predicted Age:", predicted_age)


Predicted Age: 74.74285125732422
